In [22]:
import requests
from requests.structures import CaseInsensitiveDict
import json
import time
from strava_key_gen import access_token
import os 
from typing import Dict, Tuple
import urllib
import pandas as pd
import polyline
from geopy import distance
from math import sin, cos, sqrt, atan2, radians

id = 5270481


def return_segment(id: int, access_token: str) -> json:
    url = f"https://www.strava.com/api/v3/segments/{id}"

    headers = CaseInsensitiveDict()
    headers["Authorization"] = f"Bearer {access_token}"

    resp = requests.get(url, headers=headers)

    return resp.json()


def get_params_from_segment_from_json(segment):
    return segment['name'], segment['average_grade'], segment['maximum_grade'], polyline.decode(segment['map']['polyline'])

def retrieve_segment(id: int, access_token:str):
    seg_json = return_segment(id, access_token)
    name, grade, maximum_grade, lat_lon =get_params_from_segment_from_json(seg_json)
    print(f"fetching segment details for {name}")
    return name, grade, maximum_grade, lat_lon


def make_remote_request(url: str, params: dict):

   count = 1
   while True:
       try:
           response = requests.get((url + urllib.parse.urlencode(params)))
       except (OSError, urllib3.exceptions.ProtocolError) as error:
           print('\n')
           print('*' * 20, 'Error Occured', '*' * 20)
           print(f'Number of tries: {count}')
           print(f'URL: {url}')
           print(error)
           print('\n')
           count += 1
           continue
       break

   return response

def elevation_function(lat:float, lon:float):
   time.sleep(1)
   url = 'https://api.opentopodata.org/v1/eudem25m?'
   params = {'locations': f"{lat},{lon}"}
   result = make_remote_request(url, params)
   return result.json()['results'][0]['elevation']

def get_distance_delta(point1: list, point2: list) -> float:
    return distance.distance(point1, point2).km * 1000 # return delta in meters 

def get_elevation_delta(point1: list, point2:list) -> float:
    point1_elev = elevation_function(point1[0], point1[1])
    point2_elev = elevation_function(point2[0], point2[1])
    return point2_elev - point1_elev

def get_run_rise_grade(point1:list, point2:list) -> Tuple[float, float, float]:
    run  = get_distance_delta(point1, point2)
    rise = get_elevation_delta(point1, point2)
    return run, rise, rise/run *100 

In [45]:
def get_elevations_lat_lon(lat_lon:list):
    string = ''
    for i in range(len(lat_lon)):
        print(i)
        lat, lon = lat_lon[i][0], lat_lon[i][1]
        string += str(lat) + ',' + str(lon)+ '|' 
    url = f"https://api.opentopodata.org/v1/eudem25m?locations={string[:-1]}"
    resp = requests.get(url)
    resp = resp.json()['results']
    result_list = []
    for i in range(len(resp)):
        result_list.append([resp[i]['elevation'], resp[i]['location']['lat'], resp[i]['location']['lng']])
    return result_list


get_elevations_lat_lon(lat_lon=lat_lon)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136


KeyError: 'results'

In [78]:
def get_seg_details_as_list():
    retrieve_segment(id, access_token)
    storage_list = []
    print('Remotely fetching elevation data ')
    for i in range(len(lat_lon)-1):
        storage_list.append(get_run_rise_grade(lat_lon[i],lat_lon[i+1]))




IndexError: list index out of range